# First test of Caddy bot

In [3]:
import os
from langchain_ollama import ChatOllama
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage

# Initialize the two LLMs
golf_classifier_llm = ChatOllama(
    model="llama3.2:3b-instruct-fp16",
    temperature=0.0,
    max_tokens=256,
)

golf_caddy_llm = ChatOllama(
    model="llama3.2:3b-instruct-fp16",
    temperature=0.75,
    max_tokens=1000,
)




In [4]:
# List of predefined situations
situations = [
    "BALL ABOVE FEET",
    "BALL BELOW FEET",
    "GREEN SIDE BUNKER SHOT",
    "DOWNHILL LIE",
    "UPHILL LIE",
    "BALL IN ROUGH",
    "FAIRWAY BUNKER",
]

def read_context_from_files(matched_situations):
    context = "Context:"
    for situation in matched_situations:
        file_name = f"{situation.replace(' ', '_')}.txt"  # Replace spaces with underscores to match file names
        file_name = 'documents/'+ file_name.lower() 
        if os.path.exists(file_name):
            with open(file_name, "r") as file:
                context += file.read() + "\n"  # Append the content with a newline
        else:
            context += f"[Info for {situation} not available]\n"
            
    return context

In [9]:



def golf_chatbot_with_context():
    # Define system messages for each LLM
    classifier_system_message = SystemMessage(
        content="""You are a golf expert specialized in classifying ball positions based on the situation described by the user. Your task is to analyze the user’s message and categorize the ball’s position into one or more of the following predefined categories:
	•	BALL ABOVE FEET
	•	BALL BELOW FEET
	•	GREEN SIDE BUNKER SHOT
	•	DOWNHILL LIE
	•	UPHILL LIE
	•	BALL IN ROUGH
	•	FAIRWAY BUNKER

Your response should only include the relevant category or categories. Do not provide advice, explanations, or address requests for help—just classify the situation based solely on the user’s description."""
    )

    caddy_system_message = SystemMessage(
        content="""You are a golf caddy, dedicated to helping me lower my golf scores. As a golf expert, your responses should be concise, focused, and strictly related to golf. If you encounter a question or situation outside your expertise, respond by stating that you do not know. Always prioritize actionable insights, clear advice, and strategies to enhance my game. Also, when provided context make sure to respond using the context.""",
    )

    print("Chatbot: Hello! I am your golf assistant. How can I help you? (type 'exit' to quit)")

    while True:
        # User input
        user_input = input("You: ")
        if user_input.lower() == "exit":
            print("Chatbot: Goodbye!")
            break

        # Prepare the messages for the classifier LLM
        classifier_messages = [classifier_system_message, HumanMessage(content=user_input)]

        # Get the classification from the first LLM
        classifier_response = golf_classifier_llm.invoke(classifier_messages)
        print(f"Chatbot (Classifier): {classifier_response.content}")

        # Parse the classifier response for matching situations
        matched_situations = [situation for situation in situations if situation in classifier_response.content]
        context = read_context_from_files(matched_situations)

        # Prepare the messages for the caddy LLM with context
        caddy_messages = [
            caddy_system_message,
            HumanMessage(content=f"{user_input}\nContext:\n{context}"),
        ]

        # Get the advice from the second LLM
        caddy_response = golf_caddy_llm.invoke(caddy_messages)
        print(f"Chatbot (Caddy): {caddy_response.content}")



In [10]:
golf_chatbot_with_context()

Chatbot: Hello! I am your golf assistant. How can I help you? (type 'exit' to quit)
Chatbot (Classifier): •	BALL ABOVE FEET
•	UPHILL LIE
Chatbot (Caddy): Considering your current position in the fairway and the challenges you're facing with the ball above your feet and uphill lie, here's a strategic approach to avoid bunkers:

**Aim for the Right Side**: Since the ball is above your feet, aim slightly right of your target. This will help counteract the leftward curve caused by the higher lofted club.

**Adjust Your Loft**: Take into account the higher loft from the uphill lie and adjust accordingly. If you're using a 7-iron instead of an 8-iron, this should provide the necessary extra lift to carry over or through the bunkers.

**Swing with a Full Body Turn**: Remember to maintain a full body turn throughout your swing. This will help generate power and control, allowing you to hit the ball higher and farther.

**Avoid Fighting the Slope**: By tilting back slightly and aligning your bo